In [4]:
####### This Code should not be changed except 'USE_GPU'. Please mail to T/A if you must need to change with proper description.
import torch
import torch.nn as nn
import torch.nn.functional as F 
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader, random_split
import pickle
import time
import math

from winning_set import WinningSetModule, send_next_layer


########### Change whether you would use GPU on this homework or not ############
USE_GPU = False
#################################################################################
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
class HorseDataset(Dataset):
    def __init__(self, filepath: str):
        with open(filepath, 'rb') as f_handler:
            data = pickle.load(f_handler)
            self.x_data = data['x']
            self.y_data = data['y']
        self.len = len(self.y_data)
    
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.len
      



In [5]:
class Horse_1st(nn.Module):
    def __init__(self, lane):
        super(Horse_1st, self).__init__()
        self.fc1 = nn.Linear(9*lane,100)
        self.bn1 = nn.BatchNorm1d(100)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(100,100)
        self.bn2 = nn.BatchNorm1d(100)
        self.relu2 = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.fc3 = nn.Linear(100,lane)        
        self.softmax = nn.Softmax(dim=1)
        self.lane = lane
        
    def forward(self, x):
      x = x.view(-1, 9*self.lane)
      x = self.fc1(x)
      x = self.bn1(x)
      x = self.relu1(x)
      x = self.fc2(x)
      x = self.bn2(x)
      x = self.relu2(x)
      x = self.dropout(x)
      x = self.fc3(x)
      out = self.softmax(x)

      return out

class Horse_2nd(nn.Module):
    def __init__(self, lane):
        super(Horse_2nd, self).__init__()
        self.fc1 = nn.Linear(9*lane,100)
        self.bn1 = nn.BatchNorm1d(100)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(100,100)
        self.bn2 = nn.BatchNorm1d(100)
        self.relu2 = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.fc3 = nn.Linear(100,lane) 
        self.softmax = nn.Softmax(dim=1)
        self.lane = lane

    def forward(self, x):
      x = x.view(-1, 9*self.lane)
      x = self.fc1(x)
      x = self.bn1(x)
      x = self.relu1(x)
      x = self.fc2(x)
      x = self.bn2(x)
      x = self.relu2(x)
      x = self.dropout(x)
      x = self.fc3(x)
      out = self.softmax(x)

      return out
    
class Horse_3rd(nn.Module):
    def __init__(self, lane):
        super(Horse_3rd, self).__init__()
        self.fc1 = nn.Linear(9*lane,100)
        self.bn1 = nn.BatchNorm1d(100)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(100,100)
        self.bn2 = nn.BatchNorm1d(100)
        self.relu2 = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.fc3 = nn.Linear(100,lane)
        self.softmax = nn.Softmax(dim=1)
        self.lane = lane

    def forward(self, x):
      x = x.view(-1, 9*self.lane)
      x = self.fc1(x)
      x = self.bn1(x)
      x = self.relu1(x)
      x = self.fc2(x)
      x = self.bn2(x)
      x = self.relu2(x)
      x = self.dropout(x)
      x = self.fc3(x)
      out = self.softmax(x)

      return out

In [6]:

def train(train_loader, model, rank, lane, criterion, optimizer, num_epochs): 
    model.train()
    for epoch in range(num_epochs):
        for k in range(7):
            for i, (x, y) in enumerate(train_data_saved[k]):
                rank_horse_list = []
                
                for batch_no in range(len(y)):
                    founded = 0
                    rank_horse = []
                    for horse_no in range(lane):
                        if y[batch_no][horse_no] == rank:
                            rank_horse.append(1)
                            founded = 1
                        else:
                            rank_horse.append(0)
                            
                    rank_horse_list.append(rank_horse)
                    
                rank_horse_list = torch.FloatTensor(rank_horse_list)

                x = x.to(device)
                y = y.to(device)

                optimizer.zero_grad()
                outputs = model(x)
                loss = criterion(outputs, rank_horse_list)
                loss.backward()
                optimizer.step()

        if (epoch+1)%5 == 0:
            print("Epoch [%d/%d], Loss: %.4f"
                  %(epoch+1, num_epochs, loss.data))

In [7]:
def test(test_loader, model, rank, lane):
    model.eval()
    correct = 0
    total = 0
    for x, y in test_loader:
        rank_horse_list = []
        """
        for batch_no in range(len(y)):
            founded = 0
            for horse_no in range(7):
                if y[batch_no][horse_no] == rank:
                    rank_horse_list.append(horse_no)
                    founded = 1
                    break
            if not founded:
                for horse_no in range(7):
                    if y[batch_no][6-horse_no] == rank-1:
                        rank_horse_list.append(6-horse_no)
                        break
        """
        for batch_no in range(len(y)):
            rank_horse = []
            for horse_no in range(lane):
                if y[batch_no][horse_no] == rank:
                    rank_horse.append(1)
                else:
                    rank_horse.append(0)

            rank_horse_list.append(rank_horse)        
        
        rank_horse_list = torch.FloatTensor(rank_horse_list)
        
        x = x.to(device)

        inference_outputs = model(x)
        #print(inference_outputs)
        _, predicted = torch.max(inference_outputs, 1)
        _, real = torch.max(rank_horse_list, 1)
        total += x.size(0)
        correct += (predicted == real).sum()
        #print(predicted, rank_horse_list)

    correct = torch.tensor(correct, dtype=torch.float32, device=device)
    total = torch.tensor(total, dtype=torch.float32, device=device)

    print("Accuacy of the model on test:%.4f %%" % (100.0*correct/total))
    #print(total)


In [9]:
criterion = nn.MSELoss()
num_epochs = 100

model_1st = Horse_1st(10)
model_1st.to(device)

optimizer = torch.optim.Adam(model_1st.parameters(), lr=0.001)

train(train_loader, model_1st, 1, 10, criterion, optimizer, num_epochs)

test(test_loader, model_1st, 1, 10)

NameError: name 'train_loader' is not defined

In [10]:
model_2nd = Horse_2nd(10)
model_2nd.to(device)

optimizer = torch.optim.Adam(model_2nd.parameters(), lr=0.001)

train(train_loader, model_2nd, 2, 10, criterion, optimizer, num_epochs)

test(test_loader, model_2nd, 2, 10)

NameError: name 'train_loader' is not defined

In [11]:
model_3rd = Horse_3rd(10)
model_3rd.to(device)

optimizer = torch.optim.Adam(model_3rd.parameters(), lr=0.001)

train(train_loader, model_3rd, 3, 10, criterion, optimizer, num_epochs)

test(test_loader, model_3rd, 3, 10)

NameError: name 'train_loader' is not defined

In [12]:
def predict_Top3(test_loader, lane, model_1st, model_2nd, model_3rd):
    model_1st.eval()
    model_2nd.eval()
    model_3rd.eval()
    
    total = 0
    sum_1st = 0.
    sum_1st2nd = 0.
    correct_1 = 0
    correct_12 = 0
    correct_123 = 0
    
    for x, y in test_loader:
        # change start
        # x, y_label = send_next_layer(cand_model, x)
        predict_list_1st = model_1st(x)
        predict_list_2nd = model_2nd(x)
        predict_list_3rd = model_3rd(x)
        
        prob_list_1st, predict_list_1st = torch.sort(predict_list_1st, descending=True)
        prob_list_2nd, predict_list_2nd = torch.sort(predict_list_2nd, descending=True)
        prob_list_3rd, predict_list_3rd = torch.sort(predict_list_3rd, descending=True)
        
        prob_1st = prob_list_1st[:,0] 
        prob_2nd = prob_list_2nd[:,0] 
        prob_3rd = prob_list_3rd[:,0] 
        
        predicted_1st = predict_list_1st[:,0]
        predicted_2nd = predict_list_2nd[:,0]
        predicted_3rd = predict_list_3rd[:,0]
        """
        predicted_1st = [y_label[index][val] for (index, val) in enumerate(predict_list_1st[:,0])]
        predicted_2nd = [y_label[index][val] for (index, val) in enumerate(predict_list_2nd[:,0])]
        predicted_3rd = [y_label[index][val] for (index, val) in enumerate(predict_list_3rd[:,0])]
        """
        # change end

        
        for batch_no in range(len(y)):
            if predicted_1st[batch_no] == predicted_2nd[batch_no]:
                if prob_list_1st[batch_no][0]*prob_list_2nd[batch_no][1] > prob_list_1st[batch_no][1]*prob_list_2nd[batch_no][0]:
                    prob_2nd[batch_no] = prob_list_2nd[batch_no][1]
                    predicted_2nd[batch_no] = predict_list_2nd[batch_no][1]
                else:
                    prob_1st[batch_no] = prob_list_1st[batch_no][1]
                    predicted_1st[batch_no] = predict_list_1st[batch_no][1]

            if predicted_2nd[batch_no] == predicted_3rd[batch_no]:
                if prob_list_2nd[batch_no][0]*prob_list_3rd[batch_no][1] > prob_list_2nd[batch_no][1]*prob_list_3rd[batch_no][0]:
                    prob_3rd[batch_no] = prob_list_3rd[batch_no][1]
                    predicted_3rd[batch_no] = predict_list_3rd[batch_no][1]
                else:
                    prob_2nd[batch_no] = prob_list_2nd[batch_no][1]
                    predicted_2nd[batch_no] = predict_list_2nd[batch_no][1]
        #print(prob_list_3rd[:,0])
                
        for batch_no in range(len(y)):
            for horse_no in range(lane):
                if y[batch_no][horse_no] == 1:
                    real_1st = horse_no
                elif y[batch_no][horse_no] == 2:
                    real_2nd = horse_no
                elif y[batch_no][horse_no] == 3:
                    real_3rd = horse_no
            """
            if predicted_1st == predicted_2nd:
                if prob_list_1st[0]*prob_list_2nd[1] > prob_list_1st[1]*prob_list_2nd[0]:
                    prob_2nd = prob_list_2nd[1]
                    predicted_2nd = predict_list_2nd[1]
                else:
                    prob_1st = prob_list_1st[1]
                    predicted_1st = predict_list_1st[1]
            """   
            if predicted_1st[batch_no] == real_1st:
                #print("단승 예측: %d, 확률: %.1f, 성공" %(predicted_1st[batch_no], 100.0*prob_1st[batch_no]))
                correct_1 += 1
            else:
                #print("단승 예측: %d, 확률: %.1f, 실패" %(predicted_1st[batch_no], 100.0*prob_1st[batch_no]))
                correct_1 += 0
            
            if predicted_1st[batch_no] == real_1st and predicted_2nd[batch_no] == real_2nd:
                #print("쌍승 예측: (%d, %d), 확률: %.1f, 성공" %(predicted_1st[batch_no], predicted_2nd[batch_no], 100.0*prob_1st[batch_no]*prob_2nd[batch_no]))              
                correct_12 += 1
            else:
                #print("쌍승 예측: (%d, %d), 확률: %.1f, 실패" %(predicted_1st[batch_no], predicted_2nd[batch_no], 100.0*prob_1st[batch_no]*prob_2nd[batch_no]))    
                correct_12 += 0      
                
            if predicted_1st[batch_no] == real_1st and predicted_2nd[batch_no] == real_2nd and predicted_3rd[batch_no] == real_3rd:
#                 print("삼쌍승 예측: (%d, %d, %d), 확률: %.1f, 성공" %(predicted_1st[batch_no], predicted_2nd[batch_no], predicted_3rd[batch_no], 100.0*prob_1st[batch_no]*prob_2nd[batch_no]*prob_3rd[batch_no]))              
                correct_123 += 1
            else:
#                 print("삼쌍승 예측: (%d, %d, %d), 확률: %.1f, 실패" %(predicted_1st[batch_no], predicted_2nd[batch_no], predicted_3rd[batch_no], 100.0*prob_1st[batch_no]*prob_2nd[batch_no]*prob_3rd[batch_no]))    
                correct_123 += 0            
            

            
        total += len(x)

    print(f'단승: {correct_1}, 연승: {correct_12}, 삼쌍승: {correct_123}, 총 개수: {total}')


In [13]:
predict_Top3(test_loader, 10, model_1st, model_2nd, model_3rd)

NameError: name 'test_loader' is not defined

In [14]:
# torch.save(model_1st.state_dict(), '1st_model_pt.pt')
# torch.save(model_2nd.state_dict(), '2nd_model_pt.pt')
# torch.save(model_3rd.state_dict(), '3rd_model_pt.pt')

In [15]:
model_1st.load_state_dict(torch.load('1st_model_pt.pt', map_location=device))
model_2nd.load_state_dict(torch.load('2nd_model_pt.pt', map_location=device))
model_3rd.load_state_dict(torch.load('3rd_model_pt.pt', map_location=device))

RuntimeError: Error(s) in loading state_dict for Horse_1st:
	size mismatch for fc1.weight: copying a param with shape torch.Size([100, 63]) from checkpoint, the shape in current model is torch.Size([100, 90]).
	size mismatch for fc3.weight: copying a param with shape torch.Size([7, 100]) from checkpoint, the shape in current model is torch.Size([10, 100]).
	size mismatch for fc3.bias: copying a param with shape torch.Size([7]) from checkpoint, the shape in current model is torch.Size([10]).

In [16]:
cand_model = WinningSetModule(9, 8, 14)
cand_model.load_state_dict(torch.load('fng_pt2.pt', map_location=device))
cand_model = cand_model.to(device=device)

for lane in range(7, 15):
    dataset = HorseDataset(f'./preprocess/data/data_{lane:02}.pkl')
    test_loader = DataLoader(dataset=dataset, batch_size=20, shuffle=False)
    print(f'lane 수: {lane}')
    predict_Top3(test_loader, cand_model, model_1st, model_2nd, model_3rd)

lane 수: 7


IndexError: index 14 is out of bounds for dimension 0 with size 14

In [ ]:
cand_model = WinningSetModule(9, 8, 14)
cand_model.load_state_dict(torch.load('fng_pt2.pt', map_location=device))
cand_model = cand_model.to(device=device)

print('lane 수: 7')
predict_Top3(test_loader, cand_model, model_1st, model_2nd, model_3rd)

for lane in range(8, 15):
    dataset = HorseDataset(f'./preprocess/data/data_{lane:02}.pkl')
    test_loader = DataLoader(dataset=dataset, batch_size=20, shuffle=False)
    print(f'lane 수: {lane}')
    predict_Top3(test_loader, cand_model, model_1st, model_2nd, model_3rd)

In [1]:
print(test_data_saved)
print(train_data_saved)


NameError: name 'test_data_saved' is not defined

In [17]:
print('using device:', device)
loss_fcn = nn.MSELoss()

model = WinningSetModule(9, 8, 14)
_size = 8

n_iters = 30

test_data_saved = []
train_data_saved = []

for _size in range(8, 15):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    dataset = HorseDataset(f'./preprocess/data/data_{_size:02}.pkl')
    _dataset_size = len(dataset)
    _test_data_size = _dataset_size // 10
    _train_data_size = _dataset_size - _test_data_size
    
    #print([_train_data_size, _test_data_size])
    
    train_dataset, test_dataset = random_split(dataset, [_train_data_size, _test_data_size])
    
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
    w_train(train_loader, model, optimizer, n_iters)
    
    test_data_saved.append(test_loader) # test를 추출합니다
    train_data_saved.append(train_loader) # train를 추출합니다

    s = 0
    total = 0
    for i, (x, y) in enumerate(test_loader):
        x = x.to(device=device)
        y = -y
        y = y.to(device=device)
        output = model(x)
        predicted = torch.topk(output, 7, sorted=False)[1]
        sc = 7
        result = torch.topk(y, sc, sorted=False)[1]
        s += sum([sum([1 if (val in result[index]) else 0 for val in predicted[index]]) for index in range(len(x))]) / sc
        total += len(y)
    print(f'lane size: {_size}, top 7 accuarcy_1: {s / total}%, {total}')

    s = 0
    total = 0
    for i, (x, y) in enumerate(test_loader):
        x = x.to(device=device)
        y = -y
        y = y.to(device=device)
        output = model(x)
        predicted = torch.topk(output, 7, sorted=False)[1]
        sc = 7
        result = torch.topk(y, sc, sorted=False)[1]
        s += sum([1 if (sum([1 if (val in result[index]) else 0 for val in predicted[index]]) == sc) else 0 for index in range(len(x))])
        total += len(y)
    print(f'lane size: {_size}, top 7 accuarcy_2: {s / total}%, {total}')
    
    s = 0
    total = 0
    for i, (x, y) in enumerate(test_loader):
        x = x.to(device=device)
        y = -y
        y = y.to(device=device)
        output = model(x)
        predicted = torch.topk(output, 7, sorted=False)[1]
        sc = 3
        result = torch.topk(y, sc, sorted=False)[1]
        s += sum([sum([1 if (val in result[index]) else 0 for val in predicted[index]]) for index in range(len(x))]) / sc
        total += len(y)
    print(f'lane size: {_size}, top 3 accuarcy_1: {s / total}%, {total}')
    
    s = 0
    total = 0
    for i, (x, y) in enumerate(test_loader):
        x = x.to(device=device)
        y = -y
        y = y.to(device=device)
        output = model(x)
        predicted = torch.topk(output, 7, sorted=False)[1]
        sc = 3
        result = torch.topk(y, sc, sorted=False)[1]
        s += sum([1 if (sum([1 if (val in result[index]) else 0 for val in predicted[index]]) == sc) else 0 for index in range(len(x))])
        total += len(y)
    print(f'lane size: {_size}, top 3 accuarcy_2: {s / total}%, {total}')


using device: cpu


TypeError: train() missing 3 required positional arguments: 'criterion', 'optimizer', and 'num_epochs'

In [ ]:
'''
if __name__ == '__main__':
    dataset = HorseDataset('./preprocess/data/data_10.pkl')

    

    train_size = int(0.8*len(dataset))
    test_size = len(dataset)-train_size
    
    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
    
    train_loader = DataLoader(dataset=train_dataset, batch_size=20, shuffle=True)
    test_loader = DataLoader(dataset=test_dataset, batch_size=20, shuffle=False)
    
    for i, (x, y) in enumerate(train_loader):
        print(x.shape)
#         print(y.shape)
#     for i, (x, y) in enumerate(test_loader):
#         print(x.shape)
#         print(y.shape)
'''